## Euler's integration

In [ ]:
# Init simulation

import pygame
from matplotlib import pyplot as plt

from physics import kinetic, universe, universe_utils
from graphics import manager

pygame.display.set_mode = lambda *_: None
kinetic.Spawner.tick = lambda *_ : None

mgr = manager.Manager()

TICKS = 200
DT = 5

uni = universe.Universe()

In [ ]:
import unifile
unifile.initialize()
sun = uni.kinetic_registry._Registry__registry[0]

x = []
planets = [ planet.name for planet in uni.kinetic_registry ]
y = [ [] for _ in uni.kinetic_registry ]
ax = [ [] for _ in uni.kinetic_registry ]
ay = [ [] for _ in uni.kinetic_registry ]
dr = [ [] for _ in uni.kinetic_registry ]
E = [ [] for _ in uni.kinetic_registry ]

for tick in range(TICKS):
    for i in range(len(uni.kinetic_registry)):
        planet = uni.kinetic_registry._Registry__registry[i]
        y[i].append(planet.current_velocity.magnitude)
    
        ax[i].append(planet.current_velocity.x)
        ay[i].append(planet.current_velocity.y)
        Ek = (planet.current_velocity.magnitude ** 2) / 2
        Ep = universe_utils.distance(planet, sun)
        E[i].append(Ek + Ep)
        dr[i].append(universe_utils.distance(planet, sun))
    x.append(DT * tick)
    uni.tick(DT)

In [ ]:
for Ep in E[1:]:
    if len(x) != len(Ep):
        continue
    avg = sum(Ep) / len(Ep)
    err = [ ((energy / avg) - 1) * 100 for energy in Ep ]
    plt.plot(x, err, label=f"{planets[E.index(Ep)]}")
    
plt.legend()
plt.title(f"System energy error over time")
plt.xlabel('Time')
plt.ylabel('drift, %')
plt.show()

plt.plot(x, y[3])
plt.plot(x, ax[3])
plt.plot(x, ay[3])
plt.xlabel('Time')
plt.ylabel('Velocity (magnitude, x, y)')

plt.title('Earth velocity over time (magnitude, velocity.x, velocity.y)')

plt.show()

plt.plot(x, dr[3])

aphelion = dr[3].index(max(dr[3]))
perihelion = dr[3].index(min(dr[3]))

plt.plot(x[aphelion], dr[3][aphelion], 'ro')
plt.annotate(str(dr[3][aphelion] / 1e6), (x[aphelion], dr[3][aphelion]))
plt.plot(x[perihelion], dr[3][perihelion], 'ro')
plt.annotate(str(dr[3][perihelion] / 1e6), (x[perihelion], dr[3][perihelion]))
plt.title('Earth orbit params (aphelion, perihelion), m')
plt.show()

# Plotting data

In [ ]:
x = []
y = [ [] for _ in uni.kinetic_registry ]
ax = [ [] for _ in uni.kinetic_registry ]
ay = [ [] for _ in uni.kinetic_registry ]
dr = [ [] for _ in uni.kinetic_registry ]
E = [ [] for _ in uni.kinetic_registry ]

DT = 1

for tick in range(TICKS):
    for i in range(len(uni.kinetic_registry)):
        planet = uni.kinetic_registry._Registry__registry[i]
        y[i].append(planet.current_velocity.magnitude)
    
        ax[i].append(planet.current_velocity.x)
        ay[i].append(planet.current_velocity.y)
        Ek = (planet.current_velocity.magnitude ** 2) / 2
        Ep = universe_utils.distance(planet, sun)
        E[i].append(Ek + Ep)
        dr.append(universe_utils.distance(planet, sun))
    x.append(DT)
    uni.tick(DT)
    DT += 0.5

In [ ]:
for Ep in E[1:]:
    if len(x) != len(Ep):
        continue
    avg = sum(Ep) / len(Ep)
    err = [ ((energy / avg) - 1) * 100 for energy in Ep ]
    plt.plot(x, err, label=f"{planets[E.index(Ep)]}")
    
plt.legend()
plt.title(f"System energy error over time (variable DT")
plt.xlabel('Time')
plt.ylabel('drift, %')
plt.show()

# Plotting data

In [ ]:
uni.finalize()

# Cleanup all kinetics, restart the universe